<a href="https://colab.research.google.com/github/veruizr/ML_Doc/blob/main/apriori_FPgrowth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
## Algoritmo apriori

import polars as pl

# Crear el DataFrame con los datos de ejemplo
data = {
    "cliente": [1, 2, 3, 4, 5, 6],
    "productos": ["a,b,c", "a,b", "a,c,d", "a", "b,c", "b,c,d"]
}

df = pl.DataFrame(data)
print("Datos:")
print(df)

# Convertir las cadenas de productos en listas
df=df.with_columns(
    pl.col("productos").str.split(",")
)

# Obtener la lista de todas las transacciones
transacciones = df["productos"].to_list()
print("\nTransacciones:")
print(transacciones)
from itertools import combinations
from collections import defaultdict

#2. revisar items frecuentes
def encontrar_itemsets_frecuentes(transacciones, min_soporte):
    # Contar items individuales
    conteo_items = defaultdict(int)
    for trans in transacciones:
        for item in trans:
            conteo_items[frozenset([item])] += 1

    num_transacciones = len(transacciones)
    itemsets_frecuentes = []
    itemsets_actual = {}

    # Filtrar items individuales por soporte mínimo
    for item, count in conteo_items.items():
        soporte = count / num_transacciones
        if soporte >= min_soporte:
            itemsets_actual[item] = count
            itemsets_frecuentes.append((item, soporte))

    k = 2  # Tamaño del itemset
    while itemsets_actual:
        itemsets_candidatos = generar_candidatos(itemsets_actual.keys(), k)
        conteo_itemsets = defaultdict(int)

        # Contar ocurrencias de itemsets candidatos
        for trans in transacciones:
            trans_set = set(trans)
            for cand in itemsets_candidatos:
                if cand.issubset(trans_set):
                    conteo_itemsets[cand] += 1

        itemsets_actual = {}
        for itemset, count in conteo_itemsets.items():
            soporte = count / num_transacciones
            if soporte >= min_soporte:
                itemsets_actual[itemset] = count
                itemsets_frecuentes.append((itemset, soporte))

        k += 1

    return itemsets_frecuentes

def generar_candidatos(itemsets, k):
    candidatos = set()
    itemsets = list(itemsets)

    # Generar combinaciones de tamaño k
    for i in range(len(itemsets)):
        for j in range(i+1, len(itemsets)):
            union = itemsets[i].union(itemsets[j])
            if len(union) == k:
                candidatos.add(union)

    return candidatos

# generar reglas de asociación
def generar_reglas(itemsets_frecuentes, min_confianza):
    reglas = []

    for itemset, soporte_itemset in itemsets_frecuentes:
        if len(itemset) > 1:
            # Generar todas las posibles reglas
            for tam_antecedente in range(1, len(itemset)):
                for antecedente in combinations(itemset, tam_antecedente):
                    antecedente = frozenset(antecedente)
                    consecuente = itemset.difference(antecedente)

                    # Encontrar soporte del antecedente
                    for ant, soporte_ant in itemsets_frecuentes:
                        if ant == antecedente:
                            confianza = soporte_itemset / soporte_ant
                            if confianza >= min_confianza:
                                lift = soporte_itemset / (soporte_ant * encontrar_soporte(consecuente, itemsets_frecuentes))
                                reglas.append((antecedente, consecuente, soporte_itemset, confianza, lift))

    return reglas

def encontrar_soporte(itemset, itemsets_frecuentes):
    for itemset_frec, soporte in itemsets_frecuentes:
        if itemset_frec == itemset:
            return soporte
    return 0.0

    # Parámetros
min_soporte = 0.5  # 50%
min_confianza = 0.7  # 70%

# Ejecutar Algoritmo
itemsets_frecuentes = encontrar_itemsets_frecuentes(transacciones, min_soporte)
reglas = generar_reglas(itemsets_frecuentes, min_confianza)

# Mostrar resultados
print("\nItemsets frecuentes:")
for itemset, soporte in itemsets_frecuentes:
    print(f"{set(itemset)}: Soporte = {soporte:.2f}")

print("\nReglas de asociación encontradas:")
for antecedente, consecuente, soporte, confianza, lift in reglas:
    print(f"{set(antecedente)} => {set(consecuente)}: "
          f"Soporte={soporte:.2f}, Confianza={confianza:.2f}, Lift={lift:.2f}")

    # Crear DataFrame de itemsets frecuentes
itemsets_df = pl.DataFrame(
    {
        "itemset": [set(itemset) for itemset, _ in itemsets_frecuentes],
        "soporte": [soporte for _, soporte in itemsets_frecuentes]
    }
).sort("soporte", descending=True)

print("\nItemsets frecuentes ordenados:")
print(itemsets_df)

# Crear DataFrame de reglas
if reglas:
    reglas_df = pl.DataFrame(
        {
            "antecedente": [set(antecedente) for antecedente, _, _, _, _ in reglas],
            "consecuente": [set(consecuente) for _, consecuente, _, _, _ in reglas],
            "soporte": [soporte for _, _, soporte, _, _ in reglas],
            "confianza": [confianza for _, _, _, confianza, _ in reglas],
            "lift": [lift for _, _, _, _, lift in reglas]
        }
    ).sort("lift", descending=True)

    print("\nReglas de asociación ordenadas por lift:")
    print(reglas_df)
else:
    print("\nNo se encontraron reglas con los parámetros dados.")



Datos:
shape: (6, 2)
┌─────────┬───────────┐
│ cliente ┆ productos │
│ ---     ┆ ---       │
│ i64     ┆ str       │
╞═════════╪═══════════╡
│ 1       ┆ a,b,c     │
│ 2       ┆ a,b       │
│ 3       ┆ a,c,d     │
│ 4       ┆ a         │
│ 5       ┆ b,c       │
│ 6       ┆ b,c,d     │
└─────────┴───────────┘

Transacciones:
[['a', 'b', 'c'], ['a', 'b'], ['a', 'c', 'd'], ['a'], ['b', 'c'], ['b', 'c', 'd']]

Itemsets frecuentes:
{'a'}: Soporte = 0.67
{'b'}: Soporte = 0.67
{'c'}: Soporte = 0.67
{'c', 'b'}: Soporte = 0.50

Reglas de asociación encontradas:
{'c'} => {'b'}: Soporte=0.50, Confianza=0.75, Lift=1.12
{'b'} => {'c'}: Soporte=0.50, Confianza=0.75, Lift=1.12

Itemsets frecuentes ordenados:
shape: (4, 2)
┌────────────┬──────────┐
│ itemset    ┆ soporte  │
│ ---        ┆ ---      │
│ object     ┆ f64      │
╞════════════╪══════════╡
│ {'a'}      ┆ 0.666667 │
│ {'b'}      ┆ 0.666667 │
│ {'c'}      ┆ 0.666667 │
│ {'c', 'b'} ┆ 0.5      │
└────────────┴──────────┘

Reglas de asociación or